In [3]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil
import numpy as np
import pandas as pd
import cv2

import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
#sys.path.append("..")
import warnings
import mlflow

#from modeling.config import EXPERIMENT_NAME
#TRACKING_URI = open("../.mlflow_uri").read().strip()

import glob
import random
# Set the seed for reproducibility
random.seed(42)

In [106]:
def view_image(img_path, title = 'Title'):
    #imgocv = cv2.imread(img_path)
    #img_rgb = cv2.cvtColor(imgocv, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(20,15))
    plt.imshow(img_path)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [44]:
path_to_yolo = '../models/yolov8x.pt'
yolo_model = YOLO(path_to_yolo)

## Train test split => PKlot_Non_Occupied_original/PUC/Sunny

In [130]:
 ''' 
# Paths and settings
processed_path = '../data/PKLot/Sunny_most_empty/'

original_path = os.path.join(processed_path,'labels_txt')
image_source = os.path.join(processed_path, 'images')

train_path = os.path.join(processed_path, 'train')
val_path = os.path.join(processed_path, 'val')
test_path = os.path.join(processed_path, 'test')

#image_width = 1280  # replace with actual image width - done
#image_height = 720  # replace with actual image height - done
#selected_class = 0  # Choose 0 or 1 depending on the object class you want to process - currently use only empty spots

'''

In [131]:
def save_files(files, im_source, destination_images, destination_labels):
    for file_path in files:
        # Extract file name without extension
        file_name = os.path.splitext(os.path.basename(file_path))[0]

        # Define paths for images and labels
        image_source_path = os.path.join(im_source, f"{file_name}.jpg")  # Assuming images have the same name as the labels with a different extension
        label_source_path = file_path

        image_destination_path = os.path.join(destination_images, f"{file_name}.jpg")
        label_destination_path = os.path.join(destination_labels, f"{file_name}.txt")

        # Copy or move image and label files to the destination
        shutil.copy(image_source_path, image_destination_path)  # Change to shutil.move if you want to move instead of copy
        shutil.copy(label_source_path, label_destination_path)

In [132]:
# Create necessary directories
'''
os.makedirs(os.path.join(train_path, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_path, 'labels'), exist_ok=True)
os.makedirs(os.path.join(val_path, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_path, 'labels'), exist_ok=True)
os.makedirs(os.path.join(test_path, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_path, 'labels'), exist_ok=True)

'''

In [133]:
'''
# Get all txt files and split them
txt_files = glob.glob(os.path.join(original_path, '*.txt'))
random.shuffle(txt_files)

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_split = int(train_ratio * len(txt_files))
val_split = int((train_ratio + val_ratio) * len(txt_files))

train_files = txt_files[:train_split]
val_files = txt_files[train_split:val_split]
test_files = txt_files[val_split:]

# Process training, validation, and test files (same as before, but now includes selected_class)
save_files(train_files, image_source, os.path.join(train_path, 'images'), os.path.join(train_path, 'labels'))
save_files(val_files, image_source, os.path.join(val_path, 'images'), os.path.join(val_path, 'labels'))
save_files(test_files, image_source, os.path.join(test_path, 'images'), os.path.join(test_path, 'labels'))

'''



## Make the puc_sunny.yaml

In [49]:
# Make this file manually in the following format
''' 
path = '../data/PKLot/PKLot_Non_Occupied/PUC/Sunny/

Format:
-------------
train: ../train/images
val: ../val/images

nc: 2
names: ['cat','dog']

--------------
'''

" \npath = '../data/PKLot/PKLot_Non_Occupied/PUC/Sunny/\n\nFormat:\n-------------\ntrain: ../train/images\nval: ../val/images\n\nnc: 2\nnames: ['cat','dog']\n\n--------------\n"

## Training phase of YOLO 

In [4]:
yaml_path = 'only_empty.yaml'

In [57]:
#%pip install mlflow

In [5]:
yolo8x_train_pretrained_model = YOLO('../runs/detect/train54/weights/last.pt')

In [12]:
yolo8x_train_pretrained_model.train(
    data=yaml_path, 
    epochs=2, 
    batch=16,
    translate = 0.1,  # image translation (+/- fraction)
    augment = True,
    scale = 0.2,  # image scale (+/- gain)
    shear = 0.2,  # image shear (+/- deg) from -0.5 to 0.5
    #perspective = 0.1,  # image perspective (+/- fraction), range 0-0.001
    flipud = 0.7,  # image flip up-down (probability)
    fliplr = 0.5,  # image flip left-right (probability)
    mosaic = 0.3,  # image mosaic (probability)
    mixup = 0.1, # image mixup (probability) ),
)

New https://pypi.org/project/ultralytics/8.1.4 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.1 🚀 Python-3.11.3 torch-2.1.2 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=../runs/detect/train54/weights/last.pt, data=only_empty.yaml, epochs=2, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train64, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

train: Scanning /Users/thisal_weerasekara/neufische/datasets/train/labels... 874 images, 0 backgrounds, 0 corrupt: 100%|██████████| 874/874 [00:00<00:00, 3849.95it/s]

train: New cache created: /Users/thisal_weerasekara/neufische/datasets/train/labels.cache



val: Scanning /Users/thisal_weerasekara/neufische/datasets/val/labels... 109 images, 0 backgrounds, 0 corrupt: 100%|██████████| 109/109 [00:00<00:00, 4126.46it/s]

val: New cache created: /Users/thisal_weerasekara/neufische/datasets/val/labels.cache


...

                   all        109       5466      0.969      0.935      0.986      0.675



2 epochs completed in 2.945 hours.
Optimizer stripped from /Users/thisal_weerasekara/neufische/pklot/runs/detect/train64/weights/last.pt, 136.7MB
Optimizer stripped from /Users/thisal_weerasekara/neufische/pklot/runs/detect/train64/weights/best.pt, 136.7MB

Validating /Users/thisal_weerasekara/neufische/pklot/runs/detect/train64/weights/best.pt...
Ultralytics YOLOv8.1.1 🚀 Python-3.11.3 torch-2.1.2 CPU (Apple M1)
Model summary (fused): 268 layers, 68124531 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [04:31<00:00, 67.76s/it]


                   all        109       5466       0.87      0.887      0.935      0.635
Speed: 1.3ms preprocess, 2467.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /Users/thisal_weerasekara/neufische/pklot/runs/detect/train64
MLflow: results logged to /Users/thisal_weerasekara/neufische/pklot/runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x2b7babd10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

## Make predictions

In [90]:
folder_path = "../data/PKLot/Sunny_most_empty/test"

In [91]:
# Get a list of all files in the folder
all_files = os.listdir(os.path.join(folder_path,'images/'))

# Filter only .jpg files
jpg_files = [file for file in all_files if file.lower().endswith(".jpg")]

# Select a random .jpg file
random_image = random.choice(jpg_files)

# image
path_to_image = os.path.join(folder_path, 'images/',random_image)
#path_to_xml = os.path.join(folder_path, 'labels_xml/', random_image[])


In [92]:
# image from web
#image_random_test = cv2.imread('../data/random_test/test_car_lot_2.jpg')

In [93]:
# Make the prediction

path_to_best_trained_model = '../runs/detect/train64/weights/best.pt'
best_trained_model = YOLO(path_to_best_trained_model)

pred = best_trained_model.predict(
    source=path_to_image, 
    save= False, 
    conf= 0.3,  # object confidence threshold for detection
    iou= 0.3,    # intersection over union (IoU) threshold for NMS
)


image 1/1 /Users/thisal_weerasekara/neufische/pklot/notebooks/../data/PKLot/Sunny_most_empty/test/images/2012-12-08_13_10_08.jpg: 384x640 24 empty_carlots, 457.2ms
Speed: 1.2ms preprocess, 457.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


In [94]:
# Show the results
 
for r in pred:
    im_array = r.plot(line_width=1)  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    #im.save('results.jpg')  # save image
